In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
from itertools import zip_longest



url = 'https://www.otomoto.pl'

car_make = input("Enter car make: ")
car_model = input("Enter model: ")
year = input("Enter year: ")


driver = webdriver.Safari()
driver.get(url)

#cookies
element0 = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')))
element0.click()



#make
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div/div/main/div[2]/article/article/fieldset/form/section[1]/div[1]/div[1]/div/input'))
).click()
make2 = driver.find_element(By.XPATH, f'//*[@id="{car_make.lower()}"]')
make2.click()

time.sleep(1)

#model
model1 = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div/main/div[2]/article/article/fieldset/form/section[1]/div[2]/div/div/input')
model1.click()

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f'//*[@id="{car_model.lower()}"]'))
).click()

time.sleep(1)


#year
year1 = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div/main/div[2]/article/article/fieldset/form/section[1]/div[4]/div/div/input')
year1.click()

time.sleep(1)

year2 = driver.find_element(By.XPATH, f'//*[@id="{year}"]')
year2.click()

time.sleep(1)

#show ads
element3 = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div/main/div[2]/article/article/fieldset/form/section[2]/button[1]/span/span')
element3.click()

#time.sleep(6)




#prices
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div[1]/article/section'))
)
price_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div/article/section/div[4]/div[2]/div[1]/h3')
prices = [price.text for price in price_elements]


#mileage
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div[1]/article/section'))
)
mileage_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div/article/section/div[3]/dl[1]/dd[1]')
mileage = [km.text for km in mileage_elements]


#fuel
fuel_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div/article/section/div[3]/dl[1]/dd[2]')
fuels = [fuel.text for fuel in fuel_elements]


#gearbox
gears_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div/article/section/div[3]/dl[1]/dd[3]')
gears = [gear.text for gear in gears_elements]

#year2
years_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div/article/section/div[3]/dl[1]/dd[4]')
years = [year.text for year in years_elements]





#info
info_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[2]/main/div[2]/div/div[3]/div[2]/div/article/section/div[2]/p')
infos = [info.text for info in info_elements]
capacity = []
horsepower = []
for i in infos:
    elements = i.split('•')
    elements = [element.strip() for element in elements]
    
    capacity.append(elements[0])
    horsepower.append(elements[1])

    
data = zip_longest(years, mileage, capacity, horsepower, gears, fuels, prices)

raw_df = pd.DataFrame(data, columns=['Year', 'Mileage', 'Capacity', 'Horsepower', 'Gearbox', 'Fuel', 'Price'])
driver.quit()

raw_df







Enter car make: ford
Enter model: mondeo
Enter year: 2003


In [8]:
num_cols = ['Year', 'Mileage', 'Horsepower', 'Price']

df = raw_df.copy()

df['Horsepower'] = df['Horsepower'].astype(str)
condition = df['Horsepower'].str.contains('KM')

rows_to_remove = df[~condition]
df = df.drop(rows_to_remove.index)

for col in num_cols:
    df[col] = df[col].astype(str).str.replace(r'\D', '', regex=True) #remove all non-digit characters
    df[col] = pd.to_numeric(df[col], errors='coerce') #if cannot be converted -> NaN

    
    
df = df.dropna()
df['Capacity'] = df['Capacity'].str.replace(' cm3', '', regex=False)
df['Capacity'] = df['Capacity'].str.replace(r'\s', '', regex=True) #regex -> regular expressions like r'\D' or r'\s'
df['Capacity'] = df['Capacity'].astype(int)
df['Capacity'] = (df['Capacity'] / 1000).round(1)

df['Mileage'] = df['Mileage'].astype(int)
df['Year'] = df['Year'].astype(int)

df

,Year,Mileage,Capacity,Horsepower,Gearbox,Fuel,Price
0,2020,80000,2.0,145,Automatyczna,Hybryda,130000
1,2023,10,2.0,184,Automatyczna,Hybryda Plug-in,256800
2,2023,5,2.0,184,Automatyczna,Hybryda,247600
3,2020,31075,2.0,145,Automatyczna,Hybryda,131900
4,2020,27077,1.5,193,Automatyczna,Benzyna,69900
5,2020,29250,2.0,184,Automatyczna,Hybryda,159900
6,2024,722,2.0,148,Automatyczna,Hybryda,99900
7,2021,10328,1.5,193,Automatyczna,Benzyna,72000
8,2020,61054,2.0,184,Automatyczna,Hybryda,144900
9,2023,5,2.0,148,Automatyczna,Benzyna,253600


In [9]:
df = pd.concat([df] * 10000, ignore_index=True)
df

,Year,Mileage,Capacity,Horsepower,Gearbox,Fuel,Price
0,2020,80000,2.0,145,Automatyczna,Hybryda,130000
1,2023,10,2.0,184,Automatyczna,Hybryda Plug-in,256800
2,2023,5,2.0,184,Automatyczna,Hybryda,247600
3,2020,31075,2.0,145,Automatyczna,Hybryda,131900
4,2020,27077,1.5,193,Automatyczna,Benzyna,69900
...,...,...,...,...,...,...,...
319995,2023,10,2.0,184,Automatyczna,Hybryda,253600
319996,2021,46800,2.0,146,Automatyczna,Hybryda,169850
319997,2023,10,2.0,184,Automatyczna,Hybryda,244400
319998,2020,55452,2.0,184,Automatyczna,Hybryda,154900


In [10]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
for col in list(df.select_dtypes(include='object').columns):
    df[col] = df[col].astype('category')
df.describe().T
df['Capacity'].unique()

array([2. , 1.5])

In [12]:
from sklearn.model_selection import train_test_split

target = df.pop('Price')
data = df
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.15)

print(X_train.shape)
print(X_test.shape)

(272000, 6)
(48000, 6)


In [13]:
X_train

,Year,Mileage,Capacity,Horsepower,Gearbox,Fuel
89137,2020,46000,1.5,193,Automatyczna,Benzyna
29451,2020,37836,2.0,145,Automatyczna,Benzyna
212809,2023,5,2.0,148,Automatyczna,Benzyna
94559,2023,1,2.0,184,Automatyczna,Hybryda
148629,2023,10,2.0,184,Automatyczna,Hybryda
...,...,...,...,...,...,...
222441,2023,5,2.0,148,Automatyczna,Benzyna
168836,2020,27077,1.5,193,Automatyczna,Benzyna
220556,2020,48559,2.0,184,Automatyczna,Hybryda
36328,2020,61054,2.0,184,Automatyczna,Hybryda


In [14]:
#X_train['Fuel'].unique()

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
num_cols2 = list(X_train.select_dtypes(include='number').columns)

X_train[num_cols2] = scaler.fit_transform(X_train[num_cols2]) 
X_train = pd.get_dummies(data=X_train, drop_first=True, prefix_sep='_')

X_train

,Year,Mileage,Capacity,Horsepower,Gearbox_Manualna,Fuel_Hybryda,Fuel_Hybryda Plug-in
89137,-1.173044,1.194860,-2.642244,1.232064,0,0,0
29451,-1.173044,0.823011,0.378466,-1.231125,0,0,0
212809,0.991537,-0.900094,0.378466,-1.077176,0,0,0
94559,0.991537,-0.900276,0.378466,0.770216,0,1,0
148629,0.991537,-0.899866,0.378466,0.770216,0,1,0
...,...,...,...,...,...,...,...
222441,0.991537,-0.900094,0.378466,-1.077176,0,0,0
168836,-1.173044,0.332966,-2.642244,1.232064,0,0,0
220556,-1.173044,1.311416,0.378466,0.770216,0,1,0
36328,-1.173044,1.880531,0.378466,0.770216,0,1,0


In [16]:
X_test[num_cols2] = scaler.transform(X_test[num_cols2]) 
X_test = pd.get_dummies(data=X_test, drop_first=True, prefix_sep='_')

X_test

,Year,Mileage,Capacity,Horsepower,Gearbox_Manualna,Fuel_Hybryda,Fuel_Hybryda Plug-in
227918,0.991537,-0.899866,0.378466,-1.231125,0,1,0
240587,-1.173044,0.823011,0.378466,-1.231125,0,0,0
199763,0.991537,-0.900276,0.378466,0.770216,0,1,0
210285,-0.451517,0.466101,0.378466,-1.231125,0,1,0
263327,0.991537,-0.900276,0.378466,0.770216,0,1,0
...,...,...,...,...,...,...,...
182090,-0.451517,-0.208001,0.378466,-1.231125,0,0,0
229767,-0.451517,-0.429908,-2.642244,1.232064,0,0,0
204934,1.713064,-0.867436,0.378466,-1.077176,0,1,0
309693,0.991537,-0.899866,0.378466,0.770216,0,1,0


In [17]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [18]:
# from sklearn.linear_model import Ridge

# # Ustawienie współczynnika regularyzacji alpha
# regressor = Ridge(alpha=1.0)
# regressor.fit(X_train, y_train)

In [19]:
regressor.score(X_test, y_test)

0.6395634871001723

In [20]:
y_pred = regressor.predict(X_test) #here
y_pred = np.abs(y_pred)
y_pred

array([190909.69909911, 139510.29781624, 236634.44509759, ...,
       214541.71136307, 236632.43586042, 214541.71136307])

In [21]:
intercept = regressor.intercept_
coefficients = regressor.coef_

X_train

,Year,Mileage,Capacity,Horsepower,Gearbox_Manualna,Fuel_Hybryda,Fuel_Hybryda Plug-in
89137,-1.173044,1.194860,-2.642244,1.232064,0,0,0
29451,-1.173044,0.823011,0.378466,-1.231125,0,0,0
212809,0.991537,-0.900094,0.378466,-1.077176,0,0,0
94559,0.991537,-0.900276,0.378466,0.770216,0,1,0
148629,0.991537,-0.899866,0.378466,0.770216,0,1,0
...,...,...,...,...,...,...,...
222441,0.991537,-0.900094,0.378466,-1.077176,0,0,0
168836,-1.173044,0.332966,-2.642244,1.232064,0,0,0
220556,-1.173044,1.311416,0.378466,0.770216,0,1,0
36328,-1.173044,1.880531,0.378466,0.770216,0,1,0


In [22]:
col_xtrain = X_train.columns.tolist()
col_xtrain

['Year',
 'Mileage',
 'Capacity',
 'Horsepower',
 'Gearbox_Manualna',
 'Fuel_Hybryda',
 'Fuel_Hybryda Plug-in']

In [146]:
# car_make = input("Enter car make: ")
# car_model = input("Enter model: ")
# year = input("Enter year: ")


# mileage_input = input("Enter mileage: ")
# capacity_input = input("Enter engine capacity: ")
# horsepower_input = input("Enter horsepower: ")
# gearbox_input = input("Enter gearbox: ")
# fuel_input = input("Enter fuel: ")

In [147]:

new_data = pd.DataFrame({'Year':year,
 'Mileage':200000,
 'Capacity':1.8,
 'Horsepower':110,
 'Gearbox-Manualna':1,
 'Fuel-Diesel':0,}, index=[0]) 
new_data

,Year,Mileage,Capacity,Horsepower,Gearbox-Manualna,Fuel-Diesel
0,2003,200000,1.8,110,1,0


In [148]:
new_data[['Year', 'Mileage', 'Capacity', 'Horsepower']] = scaler.transform(new_data[['Year', 'Mileage', 'Capacity', 'Horsepower']]) 

In [149]:
new_data

,Year,Mileage,Capacity,Horsepower,Gearbox-Manualna,Fuel-Diesel
0,-3.046693,-0.302766,-0.64017,-2.25678,1,0


In [150]:

#PREDICTION
predicted_price = regressor.predict(new_data)
predicted_price = np.abs(predicted_price)[0]

print(f"Predicted Price: {predicted_price:.2f} PLN")

#print(predicted_price)

Predicted Price: 3707.11 PLN
